In [4]:
import ollama
from pprint import pprint
from pydantic import BaseModel
import pandas as pd
from random import randint
import numpy as np
from tqdm.notebook import tqdm

In [10]:
query_df = pd.read_excel("./datasets/generated_data/queries_0_500.xlsx")
query_df = query_df.replace(np.nan, None)
query_df.head(5)

,id,relevant_query
0,ff9f4b9c-64f8-4b20-8cbb-698db1302f05,budget gaming desktop pc
1,ff9f4b9c-64f8-4b20-8cbb-698db1302f05,entry level desktop computer
2,ff9f4b9c-64f8-4b20-8cbb-698db1302f05,small form factor pc build
3,ff9f4b9c-64f8-4b20-8cbb-698db1302f05,gaming pc with integrated graphics
4,ff9f4b9c-64f8-4b20-8cbb-698db1302f05,AMD Ryzen 5 desktop processor


In [6]:
class Translation(BaseModel):
    translation: str

In [88]:
model = "llama3.2"

system_prompt = """You are an English to Bengali tanslator.
Translate the given product related english text into bengali. Just keep the key terms"""

In [89]:
def generate_response(model, system_prompt, prompt):
    response = ollama.generate(
        model=model,
        system=system_prompt,
        prompt=prompt,
        format=Translation.model_json_schema(),
        options={"temperature": 0.35, "top_k": 5, "top_p": 0.7, "num_predict": 2048},
    )

    try:
        query = Translation.model_validate_json(response.response)
        return query
    except Exception as e:
        print(f"Error: {e}")
    return None

In [93]:
idx = randint(0, query_df.shape[0])

query = query_df["relevant_query"][idx]
response = generate_response(model, system_prompt, query)
if response is not None:
    print(f"English query: {query}")
    print(f"Benglai query: {response.translation}")

English query: gaming laptop 2.5K display
Benglai query: গেমিং ল্যাপটপ ২.৫কে ডিসপ্লে


In [91]:
bengali_queries = {
    "id": [],
    "relevant_query": [],
}

start = 0
end = 5

for idx in tqdm(range(start, end)):
    
    id = query_df["id"][idx]
    query = query_df["relevant_query"][idx]
    response = generate_response(model, system_prompt, query)

    if response is not None:
        relevant_query = response.relevant_query
        bengali_queries["id"].extend([id] * len(relevant_query))
        bengali_queries["relevant_query"].extend(relevant_query)

    if (idx + 1) % 100 == 0: 
        print(f"Done: {idx + 1}")

  0%|          | 0/5 [00:00<?, ?it/s]

AttributeError: 'Translation' object has no attribute 'relevant_query'